In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
import re
from transformers import AdamWeightDecay
from datasets import Dataset
import math
import numpy as np
from transformers import DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
import torch
import matplotlib.pyplot as plt
import evaluate
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
import torch

2023-11-21 09:44:25.576254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 09:44:29.211321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
def json_to_dataframe(filename):
    f = open(filename)
    df = pd.read_json(f)
    print(df.iloc[0])
    f.close()
    #df["maskedd headline"]= df.apply(lambda x: x['masked headline'].replace('____', str(x['ans'])), axis=1)
    print(df.iloc[0])
    df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
    df['masked headline'] = df['masked headline'].str.replace('____', '<extra_id_0> ') + ' </s>'
    df['text'] = df[['news', 'masked headline']].apply(" ".join, axis=1)
    df = df.astype(str)
    return df

In [4]:
train_data = json_to_dataframe('Train_Numerical_Reasoning.json')

news               (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...
masked headline    ____K Walmart Part-Timers to Lose Health Insur...
calculation                                     Paraphrase(30,000,K)
ans                                                               30
Name: 0, dtype: object
news               (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...
masked headline    ____K Walmart Part-Timers to Lose Health Insur...
calculation                                     Paraphrase(30,000,K)
ans                                                               30
Name: 0, dtype: object


In [5]:
train_data.loc[train_data['calculation']=="Paraphrase(650,000,K)"]

,news,masked headline,calculation,ans,text
3513,"A Florida man has won a $650,000 settlement f...",Man Gets $<extra_id_0> K After Stripper Kick </s>,"Paraphrase(650,000,K)",650,"A Florida man has won a $650,000 settlement f..."


In [6]:
train_data.head()

,news,masked headline,calculation,ans,text
0,"As of Jan. 1, Walmart will no longer offer 30...",<extra_id_0> K Walmart Part-Timers to Lose Hea...,"Paraphrase(30,000,K)",30,"As of Jan. 1, Walmart will no longer offer 30..."
1,Dax Shepard and Kristen Bell got married at t...,Dax Shepard: Wedding to Kristen Bell Cost $<ex...,Copy(142),142,Dax Shepard and Kristen Bell got married at t...
2,"Nancy Reagan, the helpmate, backstage adviser...",Nancy Reagan Dead at <extra_id_0> </s>,Copy(94),94,"Nancy Reagan, the helpmate, backstage adviser..."
3,American Airlines faces FAA fines of more tha...,American Airlines Faces $<extra_id_0> M Fine f...,Copy(7),7,American Airlines faces FAA fines of more tha...
4,"Ingrid Lyne, the Seattle mom allegedly murder...",$<extra_id_0> K Raised for Kids of Mom Dismemb...,"Paraphrase(222,000,K)",222,"Ingrid Lyne, the Seattle mom allegedly murder..."


In [7]:
dataset = Dataset.from_pandas(train_data)
dataset = dataset.train_test_split(test_size=0.2)

In [8]:
model_name ="czearing/article-title-generator"

In [9]:
def preprocess_function(examples):   
    inputs = tokenizer(examples['text'],max_length=3066,  truncation=True)
    labels = tokenizer(text_target=examples["calculation"], max_length=128, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
tokenized = dataset.map(preprocess_function, batched=True, num_proc=4,
                        remove_columns=dataset['train'].column_names)

Map (num_proc=4):   0%|          | 0/16925 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4232 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                           model=model_name,
                                           return_tensors="tf")
#optimizer = AdamWeightDecay(learning_rate=1e-4, weight_decay_rate=0.01)
#optimizer = AdamWeightDecay(learning_rate=7e-5, weight_decay_rate=0.01)
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

In [14]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

2023-11-21 09:46:03.127950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:01:00.0, compute capability: 7.0
2023-11-21 09:46:03.128857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31141 MB memory:  -> device: 1, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:c1:00.0, compute capability: 7.0
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

To convert datasets to the tf.data.Dataset format 

In [15]:
tf_train_set = model.prepare_tf_dataset(
    tokenized["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
model.compile(optimizer=optimizer)

In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, batch_size=6,callbacks=[callback], epochs=10)

Epoch 1/10
1057/1057 [==============================] - 656s 596ms/step - loss: 0.6592 - val_loss: 0.3283
Epoch 2/10
1057/1057 [==============================] - 623s 589ms/step - loss: 0.3152 - val_loss: 0.2838
Epoch 3/10
1057/1057 [==============================] - 622s 588ms/step - loss: 0.2518 - val_loss: 0.2738
Epoch 4/10
1057/1057 [==============================] - 623s 589ms/step - loss: 0.2128 - val_loss: 0.2655
Epoch 5/10
1057/1057 [==============================] - 623s 589ms/step - loss: 0.1837 - val_loss: 0.2560
Epoch 6/10
1057/1057 [==============================] - 623s 590ms/step - loss: 0.1621 - val_loss: 0.2700


In [27]:
def preprocess_function2(examples):   
    inputs = tokenizer(examples['calculation'],max_length=400,  truncation=True)
    #labels = tokenizer(text_target=examples["headline"], max_length=128, truncation=True)
    labels = tokenizer(text_target=examples["ans"], max_length=50, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [28]:
tokenized2 = dataset.map(preprocess_function2, batched=True, #num_proc=4,
                        remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/16925 [00:00<?, ? examples/s]

Map:   0%|          | 0/4232 [00:00<?, ? examples/s]

In [29]:
model2 = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [30]:
tf_train_set2 = model2.prepare_tf_dataset(
    tokenized2["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set2 = model2.prepare_tf_dataset(
    tokenized2["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [31]:
model2.compile(optimizer=optimizer)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
history2 = model2.fit(x=tf_train_set2, validation_data=tf_test_set2,batch_size=6,callbacks=[callback], epochs=10)

Epoch 1/10
1057/1057 [==============================] - 130s 101ms/step - loss: 0.2175 - val_loss: 0.1428
Epoch 2/10
1057/1057 [==============================] - 103s 97ms/step - loss: 0.1436 - val_loss: 0.1318
Epoch 3/10
1057/1057 [==============================] - 103s 97ms/step - loss: 0.1250 - val_loss: 0.1212
Epoch 4/10
1057/1057 [==============================] - 102s 96ms/step - loss: 0.1115 - val_loss: 0.1176
Epoch 5/10
1057/1057 [==============================] - 102s 96ms/step - loss: 0.1010 - val_loss: 0.1163
Epoch 6/10
1057/1057 [==============================] - 103s 97ms/step - loss: 0.0933 - val_loss: 0.1183


In [18]:
f = open('Dev_Numerical_Reasoning.json')
df = pd.read_json(f)
df['news'] = df['news'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
df['masked headline'] = df['masked headline'].str.replace('____', '<extra_id_0> ') + ' </s>'
df['text'] = df[['news', 'masked headline']].apply(" ".join, axis=1)
df = df.astype(str)
#df = df.iloc[:200]
#df = df.iloc[100:500]
f.close()
test_data = Dataset.from_pandas(df)

In [19]:
def t5_predictions(data, tokenizer, model):
    text = data['text']
    headline = data['calculation']

    inputs = tokenizer.encode_plus(text, add_special_tokens=True,
                                   return_tensors="pt").input_ids
    outputs = model.generate(inputs,
                             max_length=128, num_beams=5, early_stopping=True)

    res = ''
    results = tokenizer.decode(outputs[0][1:], skip_special_tokens=False,
                               clean_up_tokenization_spaces=False)
    return results        

In [20]:
cal = []
for i in test_data:
    cal.append(t5_predictions(i, tokenizer, model))


2023-11-21 10:57:45.152467: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc13c01c810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-21 10:57:45.152520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2023-11-21 10:57:45.152526: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2023-11-21 10:57:45.156347: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-21 10:57:45.183636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-11-21 10:57:45.190202: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation o

In [21]:
updated_cal = []
for c in cal:
    updated_cal.append(c.split('<')[0])

In [32]:
def t5_predictions2(data, tokenizer, model):


    inputs = tokenizer.encode_plus(data, add_special_tokens=True,
                                   return_tensors="pt").input_ids
    outputs = model.generate(inputs,
                             max_length=64, num_beams=5, early_stopping=True)


    res = ''
    results = tokenizer.decode(outputs[0][1:], skip_special_tokens=False,
                               clean_up_tokenization_spaces=False)
    res = re.findall(r"([\d:,./]+)", results)
    return res[0]

In [34]:
false=0
pred = []
for x,y in zip(updated_cal, test_data['ans']):
    ans = t5_predictions2(x, tokenizer, model2)
    pred.append(ans)
    if ans != y:
        print(ans)
        print(y)
        print("")
        false += 1


5
9

1.29
1

1.1
2

2010
2000

3
5

2
59

4
4.5

120
44

700
350

30
31

2
47

79.5
160

70
44

1988
21

3
29

3
4

24
85

19
1892

12
17

1
6

3
2

2
12

4
26

91
62

3.3
3

8
10

10
15.5

109
100

4
2

5
6

25
27

200
800

29
28.6

12
2

3.1
3

300
230

1677
1667

5.2
20

3
1

1
1.68

7
12

513
500

90
2010

6
51

2
3

1
1981

950
900

169,000
19

10
9

30
40

10
12

12
10

47
75

5
12

6
25

11
12

40
30

5
2

2013
7

33
30

6
136

200000
2

18
21

177
72

100
31

10
12

9.5
9.8

28
73

27
27.2

10
15

50
30

50
150

23.3
23

100
25

23
1958

2012
2011

6
5

45
50

3
5,000

2009
09

8.5
5

2
2014

3
2

88
89

3
32

2
1906

2
1

12
50

4
11

204.6
205

1.5
1

50
99

25.5
25

200
221

47
150

7.7
8

10.9
11

187
2

6.2
6

6
10

1.11
1.1

56
56.2

44.7
45

40
43

13
2

10
8

73
730

50
64

989
998

10
11

4
88

1.8
1.9

4
2

48
48.7

3
4

2
40

2011
2000

300
299

45
66

2
4

480
500

3
122

2
78

5
6

5
11

99
0.99

40
39

6
2

25
70

31.31
33

4
5

11
2

800
1200

2011
1

273
270

10

In [35]:
false

305

In [36]:
1-false/len(test_data['ans'])

0.8814152410575428